In [12]:
#!/usr/bin/env python  
# -*- coding: UTF-8 -*-  
#pip install jieba

import os
import importlib,sys
importlib.reload(sys)  
#sys.setdefaultencoding('utf-8')
import jieba #结巴分词 这是一个包，在之前需要安装 此代码中式精准模式
#sklearn库对数据分析做了功能的集成 安装https://scikit-learn.org/stable/
from sklearn import datasets#datasets读取要分析的数据
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer #CountVectorizer中文文档分词后笔算每个词模型成文本向量模型 TfidfTransformer每个词权重计算
from sklearn.svm import SVC #SVM分类算法 
from sklearn.ensemble import RandomForestClassifier#随机森林分类算法
from sklearn.feature_selection import chi2,SelectKBest #特则选择——卡方chi2
import pickle

stop_words_path = "E:/Subject/Txtdatasets Analzing/ExampleCode/stopwords-zh.txt"#stopwords的内容，即后面使用的删减stopwords，此处是相对路径，可用绝对路径

#获取停用词数据集
#可手动添加一些停用词
def read_stopwordset():#读取所有行 去掉换行符
    with open(stop_words_path, "r",encoding='UTF-8') as fp: 
        lines = fp.readlines()
        words = [' ','\r\n']#words先添加了一些停用词内容
        for line in lines:
            words.append(line.strip())#将读取的内容加在Words 上
    return words

#利用结巴分词器对文本内容进行分词
#输入要分词的句子
#获得分词结果，结果以列表形式返回
def fenci(sentence): 
    wordlist = jieba.lcut(sentence,cut_all=False,HMM=True) # jieba分词的lcut精确模式
    return wordlist#返回lcut中Wordlist列表

#将模型按照model_name进行保存
def save_model(model_name,model):
    feature_path = 'E:/Subject/Txtdatasets Analzing/models/'+model_name+'.pkl'#这是一个相对路径
    with open(feature_path, 'wb') as fw:
        pickle.dump(model, fw)

#将模型按照model_name进行读取
def load_model(model_name):
    tfidftransformer_path = 'E:/Subject/Txtdatasets Analzing/models/'+model_name+'.pkl'
    return pickle.load(open(tfidftransformer_path, "rb"))
    
#获取停用词
stopwords = read_stopwordset()
#按类别获取文档内容
rawData = datasets.load_files("E:/Subject/Txtdatasets Analzing/Datacluster/train",encoding = 'gbk',decode_error='ignore')#读取文件—名称 编码 错误解决方案  按类别获取文档
print("文件读取结束")
X_train = [] #文本内容 定义
Y_train = [] #文本标签
count = [0,0,0,0,0,0,0,0,0] #控制训练集 样例处理的时候只读取十个文件即可 
#过滤文件和分词
for i in range(len(rawData.data)):
    if rawData.filenames[i][-3:]=='txt':#判断文件名后缀，是则操作
        if count[rawData.target[i]]<500:#每个类别采样量（调试可以为10
            content = rawData.data[i]#第i篇文档的内容
            result = fenci(content)#内容分词 结果是词的list
            X_train.append(' '.join(result))#变成字符串 每个词以空格形式分开——得到训练集内容
            Y_train.append(rawData.target[i])#第i篇文档类别的标签传给Y
            count[rawData.target[i]]+=1
        else:
            continue
            
print("文件预处理结束")

#将文档内容转换为词-词频的模式（文档向量模型
count_vec = CountVectorizer(min_df=0.1,stop_words=stopwords) #可以利用最大最小文档频率对词进行筛选max_df=0.6,min_df=0.2
#去除stopwords
X_train = count_vec.fit_transform(X_train)#fit_transform是CountVectorizer的方法 
#输出为文档向量模型

save_model('count_vec',count_vec.vocabulary_)#保存模型count_vec

#基于卡方的特征选择
select_chi2 = SelectKBest(chi2,k=100)#卡方只是为特征评分 SelectKBest选K个最好的特征
X_train = select_chi2.fit_transform(X_train,Y_train)
save_model('select_chi2',select_chi2)

#tfidf对词赋予权重
tfidf = TfidfTransformer()
X_train = tfidf.fit_transform(X_train)
save_model('tfidf',tfidf)
#可以由LSA处理，sklean中也有 对特征的优化处理；SVD可以矩阵分解；可以想哪一步哪个时候执行更好
print("文件特征选择结束")

#利用SVM算法对文档内容进行分类学习
clf = SVC()#SVC+SVR=SVM  分类算法训练
clf.fit(X_train, Y_train)#传入参数
save_model('svm',clf)

print("文件训练结束")

#在测试集中进行测试

rawData = datasets.load_files("E:/Subject/Txtdatasets Analzing/Datacluster/test",encoding = 'gbk',decode_error='ignore')#读取数据集
X_test = []#定义两个数列
Y_test = []
#分词
count = [0,0,0,0,0,0,0,0,0]
for i in range(len(rawData.data)):
    if rawData.filenames[i][-3:]=='txt':
        if count[rawData.target[i]]<100:
            content = rawData.data[i]
            result = fenci(content)
            X_test.append(' '.join(result))
            Y_test.append(rawData.target[i])
            count[rawData.target[i]]+=1
        else:
            continue
#确保若训练用TFIDF，则测试也要，比如特征选择、转换等
X_test = count_vec.transform(X_test)#执行
X_test = select_chi2.transform(X_test)#卡方
test_tfidf = tfidf.transform(X_test)
Y_result = clf.predict(test_tfidf)#predict SVM

#计算准确率
correct = 0.0
for i in range(len(Y_result)):
    if Y_test[i] == Y_result[i]:#遍历Y的值确定准确率
        correct+=1.0
print("在测试集中准确率为："+str(correct/len(Y_result)))


文件读取结束
文件预处理结束
文件特征选择结束


D:\Python\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


文件训练结束
在测试集中准确率为：0.6788888888888889
